# build data set for tone rec eval using network partition

first of all, <b> notice that when you go from voiced-noneut to any other data set, the accuracy will drop because of the introduction of neutral tone</b>.

0. select a tone data set (voiced-uni,whole-uni,bigram)
1. divide the tone data set into five subsets (including neutral)
2. then sample from these five to build a subset data that contains all four tones
3. build a weighted fully connected graph from this data set
4. perform thresholding and community detection
5. inspect the final resulting cluster and compute their accuracy in tone recognition.

# starting from voiced unigram dataset noneut

In [86]:
import sys

def read_csv(input_file,num_meta):
    f=open(input_file,'r').read().split('\n')
    outfile=input_file.split('.csv')[0]+'_np.csv'
    data,label,all_data=[],[],[]
    for line in f:
        if line!='':
            dl=line.split(',')
            #dl=[float(i) for i in dl]
            d=line.split(',')[:num_meta]
            #d=[float(i) for i in d]
            l=line.split(',')[num_meta:]
            data.append(d)
            assert len(l)==1
            label.append(l[0])
            all_data.append(dl)
            
            
    return data,label,all_data

In [88]:
path='tone-rec-network/'
dataset='tone-rec-network/downsample_syl_noneut.csv'
data,label,adata=read_csv(dataset,-1)

In [89]:
#divide data into tone subsets (with label)
from collections import defaultdict
tone_dict_wlabel=defaultdict(list)
for tone in set(label):
    if tone=='6':
        continue
    tone_dict_wlabel[tone]=[adata[i] for i in range(len(adata)) if label[i]==tone]

In [90]:
#data no label
from collections import defaultdict
tone_dict=defaultdict(list)
for tone in set(label):
    if tone=='6':
        continue
    tone_dict[tone]=[data[i] for i in range(len(data)) if label[i]==tone]

In [91]:
for k in tone_dict.keys():
    print k,len(tone_dict[k])

1 19556
3 8374
2 19047
4 23797


In [92]:
#randomly select 2000 from each tone cat and form a test dataset
#with label
import random
n=2000
test_data_wlabel=[]
for k in tone_dict_wlabel.keys():
    this_data=random.sample(tone_dict_wlabel[k],n)
    test_data_wlabel.extend(this_data)

#no label    
import random
n=2000
test_data=[]
for k in tone_dict.keys():
    this_data=random.sample(tone_dict[k],n)
    test_data.extend(this_data)

In [93]:
print test_data[:2]
print len(test_data)
print test_data_wlabel[:2]
print len(test_data_wlabel)


[['0.362', '0.371', '0.381', '0.39', '0.4', '0.409', '0.417', '0.424', '0.431', '0.437', '0.443', '0.449', '0.454', '0.458', '0.462', '0.464', '0.467', '0.469', '0.47', '0.472', '0.473', '0.474', '0.473', '0.471', '0.465', '0.457', '0.447', '0.434', '0.421', '0.407'], ['0.187', '0.188', '0.185', '0.119', '0.116', '0.113', '0.05', '0.047', '0.045', '0.044', '0.045', '0.047', '0.048', '0.049', '0.048', '0.045', '0.042', '0.038', '0.036', '0.037', '0.105', '0.109', '0.114', '0.123', '0.13', '0.134', '0.135', '0.132', '0.126', '0.118']]
8000
[['0.211', '0.203', '0.193', '0.18', '0.166', '0.155', '0.147', '0.142', '0.137', '0.134', '0.129', '0.123', '0.117', '0.115', '0.121', '0.13', '0.137', '0.139', '0.14', '0.142', '0.144', '0.148', '0.152', '0.156', '0.159', '0.164', '0.17', '0.178', '0.184', '0.187', '1'], ['0.332', '0.328', '0.326', '0.324', '0.323', '0.323', '0.322', '0.322', '0.322', '0.322', '0.323', '0.323', '0.324', '0.325', '0.326', '0.326', '0.327', '0.327', '0.327', '0.326', '

# data set order

 in this data, there are 2000 of each tone in the order of 2000 x 1 + 2000 x 3 + 2000 x 2 + 2000 x 4.

In [98]:
#output_path='tone_rec_test_data/'
def serialize_data(test_data):
    if len(test_data[0])==30:
        output_file=dataset.replace('.csv','_subset.csv')
    else:
        output_file=dataset.replace('.csv','_subset_wlabel.csv')
    f=open(output_file,'w').close()
    f=open(output_file,'a')
    for i in test_data:#no label only data, this is for conveniently compute distance matrix later in graph
        f.write(','.join(i)+'\n')
    f.close()

In [99]:
#write to both data files
serialize_data(test_data)
serialize_data(test_data_wlabel)

In [101]:
4000*3999/2
#32 million computations for 8000 tones
#7 million for 4000 tones
#notice that for this data set the length of subsequences is only 30 point.

7998000

8000

In [37]:
t=1
for i in range(len(test_data)):
    test_data[i].append(t)
    if (i+1)%2000 == 0:
        t+=1
    

In [38]:
print test_data[:3]

[[0.271, 0.266, 0.262, 0.258, 0.206, 0.204, 0.202, 0.15, 0.148, 0.146, 0.143, 0.141, 0.138, 0.134, 0.127, 0.112, 0.103, 0.097, 0.09, 0.084, 0.079, 0.078, 0.08, 0.083, 0.086, 0.09, 0.094, 0.103, 0.113, 0.114, 1], [0.703, 0.704, 0.704, 0.705, 0.706, 0.707, 0.708, 0.709, 0.711, 0.712, 0.714, 0.717, 0.719, 0.722, 0.725, 0.728, 0.732, 0.737, 0.743, 0.75, 0.757, 0.769, 0.778, 0.784, 0.788, 0.79, 0.791, 0.792, 0.793, 0.794, 1], [0.06, 0.056, 0.043, 0.036, 0.032, 0.028, 0.024, -0.008, -0.02, -0.029, -0.036, -0.041, -0.043, -0.043, -0.041, -0.038, -0.032, -0.02, 0.003, 0.029, 0.047, 0.056, 0.056, 0.05, 0.038, 0.024, 0.009, -0.001, 0.034, 0.085, 1]]
